In [7]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [9]:

from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.environ.get["OPENAI_API_KEY"]

# To resolve PineconeConfigurationError: You haven't specified an Api-Key.
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [10]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [12]:
extracted_data = load_pdf("../data/")

In [13]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [14]:
text_chunk = text_split(extracted_data)

In [15]:
len(text_chunk)

6983

In [22]:
! pip install sentence-transformers

     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     -------------------- ------------------- 30.7/60.6 kB 1.4 MB/s eta 0:00:01
     -------------------------------- ----- 51.2/60.6 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 60.6/60.6 kB 541.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/163.3 kB ? eta -:--:--
   ---------------------- ----------------- 92.2/163.3 kB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 163.3/163.3 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
    --------------------------------------- 0.3/10.6 MB 7.9 MB/s eta 0:00:02
   - -------------------------------------- 0.3/10.6 MB 3.5 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/10.6 MB 7.7 MB/s eta 0:00:02
   ------ --------------------------------- 1.6/10.6 MB 8.5 MB/s eta 0:00:02
   -------- ------------------------------- 2.3/10.6 MB 9.8 MB/s eta 0:00:01
   

In [16]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

c:\Users\alams\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
query_result = embeddings.embed_query("Hello World")
len(query_result)

384

In [19]:
#Initializing the Pinecone
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name="medical-chatbot"


docsearch = PineconeVectorStore.from_documents(text_chunk, embeddings, index_name=index_name)


In [20]:

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='ORGANIZATIONS\nAmerican Academy of Ophthalmology. 655 Beach Street, PO\nBox 7424, San Francisco, CA 94120-7424. <http://www.\neyenet.org>.KEY TERMS\nAllergen —A substance capable of inducing an\nallergic response.\nAllergic reaction —An immune system reaction to\na substance in the environment; symptoms\ninclude rash, inflammation, sneezing, itchy watery\neyes, and runny nose.\nConjunctiva —The mucous membrane that covers\nthe white part of the eyes and lines the eyelids.', metadata={'page': 659.0, 'source': '..\\data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf'}), Document(page_content='ORGANIZATIONS\nAmerican Academy of Ophthalmology. 655 Beach Street, PO\nBox 7424, San Francisco, CA 94120-7424. <http://www.\neyenet.org>.KEY TERMS\nAllergen —A substance capable of inducing an\nallergic response.\nAllergic reaction —An immune system reaction to\na substance in the environment; symptoms\ninclude rash, inflammation, sneezing, itchy watery\neyes, and runn

In [38]:
print(docs[0].metadata)

{'page': 659.0, 'source': '..\\data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf'}


In [21]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [22]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [28]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [29]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
# while True:
#     user_input=input(f"Input Prompt:")
#     result=qa({"query": user_input})
#     print("Response : ", result["result"])

In [31]:
user_input = "What are Allergies"
result=qa({"query": user_input})
print(result)

{'query': 'What are Allergies', 'result': 'An allergy is an immune system reaction to a substance in the environment, with symptoms including rash, inflammation, sneezing, itchy watery eyes, and a runny nose.', 'source_documents': [Document(page_content='ORGANIZATIONS\nAmerican Academy of Ophthalmology. 655 Beach Street, PO\nBox 7424, San Francisco, CA 94120-7424. <http://www.\neyenet.org>.KEY TERMS\nAllergen —A substance capable of inducing an\nallergic response.\nAllergic reaction —An immune system reaction to\na substance in the environment; symptoms\ninclude rash, inflammation, sneezing, itchy watery\neyes, and runny nose.\nConjunctiva —The mucous membrane that covers\nthe white part of the eyes and lines the eyelids.', metadata={'page': 659.0, 'source': '..\\data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf'}), Document(page_content='ORGANIZATIONS\nAmerican Academy of Ophthalmology. 655 Beach Street, PO\nBox 7424, San Francisco, CA 94120-7424. <http://www.\neyenet.org>.KEY TERMS\

In [33]:
print(result["result"])

An allergy is an immune system reaction to a substance in the environment, with symptoms including rash, inflammation, sneezing, itchy watery eyes, and a runny nose.


In [34]:
! pip install -U langchain-community